In [1]:
# This example requires the 'message_content' intent.
from influxdb_client import InfluxDBClient
import os
from dotenv import load_dotenv
import pytz
from datetime import datetime
import asyncio
import functools
import typing
load_dotenv("../.env")
INFLUXDB_TOKEN=os.environ["INFLUXDB_TOKEN"]
vietnam_timezone = pytz.timezone('Asia/Ho_Chi_Minh')
org = os.environ["INFLUXDB_ORG"]
host = os.environ["INFLUXDB_HOST"]
client = InfluxDBClient(url=host, token=INFLUXDB_TOKEN, org=org)
database="stock_data"
query_api=client.query_api()



In [9]:
query='''
import "timezone"
timezone.location(name:"America/Los_Angeles")
from(bucket: "stock_data")
    |> range(start: 0)
    |> filter(fn: (r) => r._measurement == "stock_price" and r._field=="close" and r.ticker=="NVL")
    '''
result=query_api.query(query=query).to_values()
result

[dict_values(['_result', 0, datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), datetime.datetime(2024, 5, 10, 17, 1, 10, 255617, tzinfo=tzutc()), datetime.datetime(2024, 4, 22, 7, 45, tzinfo=tzutc()), 14.9, 'close', 'stock_price', 'NVL']),
 dict_values(['_result', 0, datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), datetime.datetime(2024, 5, 10, 17, 1, 10, 255617, tzinfo=tzutc()), datetime.datetime(2024, 5, 2, 3, 45, tzinfo=tzutc()), 14.75, 'close', 'stock_price', 'NVL']),
 dict_values(['_result', 0, datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), datetime.datetime(2024, 5, 10, 17, 1, 10, 255617, tzinfo=tzutc()), datetime.datetime(2024, 5, 3, 7, 45, tzinfo=tzutc()), 14.6, 'close', 'stock_price', 'NVL']),
 dict_values(['_result', 0, datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), datetime.datetime(2024, 5, 10, 17, 1, 10, 255617, tzinfo=tzutc()), datetime.datetime(2024, 5, 6, 3, 15, tzinfo=tzutc()), 14.85, 'close', 'stock_price', 'NVL']),
 dict_values(['_result', 0, datet